In [4]:
import gzip, ujson, random, json
import toolz
import cPickle as c
import nltk.tokenize
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

def getjsontext(d):
    return [toolz.keyfilter(lambda k: k == "text", d)]

def load_json_yelp(filepath):
    with gzip.open("yelp_train_academic_dataset_review.json.gz") as f:
        yelpdata = list()
        for i in xrange(20000):
            line = f.readline().strip()
            data = json.loads(line)
            yelpdata.append(data)
    return yelpdata
    
def tokenizereview(str):
    return nltk.tokenize.sent_tokenize(str)

#data loading
filepath = 'yelp_train_academic_dataset_review.json.gz'
yelp_data = load_json_yelp(filepath)

In [3]:
print len(yelp_data)
print type(yelp_data)
est_text = yelp_data[0]['text']
tt=[test_text]
print tt
test_vec = CountVectorizer(encoding='unicode')
a=test_vec.fit_transform(tt)

#test_vec.vocabulary_

8
<type 'dict'>


KeyError: 0

In [5]:
reviews = list()
grades = list()
for i in xrange(len(yelp_data)):
    reviews.append(yelp_data[i]['text'])
    grades.append(yelp_data[i]['stars'])

In [6]:
from sklearn import cross_validation
textcount_vectorizer = CountVectorizer()

textcount_ridge = linear_model.Ridge(fit_intercept=True)

counts_pipeline = Pipeline([
       ('counts', textcount_vectorizer),
       ('ridge', textcount_ridge)
   ])

params_grid = {"counts__max_df":[0.99,0.95], "counts__min_df":[0.01,0.05], "ridge__alpha":[1,9]}

cv = cross_validation.ShuffleSplit(len(grades), n_iter=20, test_size=0.2, random_state=42)

count_cross_validation = GridSearchCV(counts_pipeline, param_grid = params_grid, cv = cv, n_jobs = 2, scoring = 'mean_squared_error')

count_cross_validation.fit(reviews, grades)

GridSearchCV(cv=ShuffleSplit(20000, n_iter=20, test_size=0.2, random_state=42),
       error_score='raise',
       estimator=Pipeline(steps=[('counts', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params={}, iid=True, n_jobs=2,
       param_grid={'ridge__alpha': [1, 9], 'counts__min_df': [0.01, 0.05], 'counts__max_df': [0.99, 0.95]},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_squared_error',
       verbose=0)

In [7]:
print count_cross_validation.best_params_

{'ridge__alpha': 9, 'counts__min_df': 0.01, 'counts__max_df': 0.99}


In [8]:
import dill
with open('test_bow','wb') as output:
    dill.dump(count_cross_validation.best_estimator_,output)

In [10]:
from data import test_json

In [12]:
print type(test_json)

<type 'list'>


In [18]:
testX=list()
for j in test_json:
    print type(j)
    testX.append(j['text'])

<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>


In [19]:
print testX

["Love it!!!!! Love it!!!!!! love it!!!!!!!   Who doesn't love Culver's!", 'Everything was great except for the burgers they are greasy and very charred compared to other stores.', 'I really like both Chinese restaurants in town.  This one has outstanding crab rangoon.  Love the chicken with snow peas and mushrooms and General Tso Chicken.  Food is always ready in 10 minutes which is accurate.  Good place and they give you free pop.', 'Above average takeout with friendly staff. The sauce on the pan fried noodle is tasty. Dumplings are quite good.', "We order from Chang Jiang often and have never been disappointed.  The menu is huge, and can accomodate anyone's taste buds.  The service is quick, usually ready in 10 minutes."]


In [22]:
best_pipe =count_cross_validation.best_estimator_

In [25]:
best_pipe.predict([testX[0]])

array([ 4.22328161])